In [1]:
import numpy as np
import time
from copy import deepcopy

# Import Qiskit classes
import qiskit
import qiskit.quantum_info as qi
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer
from qiskit.providers.aer import noise
from qiskit.compiler import assemble

# Tomography functions
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.ignis.verification.tomography import process_tomography_circuits, ProcessTomographyFitter
from qiskit.ignis.verification.tomography import gateset_tomography_circuits, GatesetTomographyFitter
import qiskit.ignis.mitigation.measurement as mc

# Auxiliary methods
from qiskit.quantum_info import Choi, Kraus
from qiskit.extensions import HGate, XGate


## Custom Gates to implement simple jump, quantum jump and quantum merge operations.

In [2]:
from qiskit import transpile

Usplit = [[1, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1.j, 0, 0, 0],
         [0, 1.j / np.sqrt(2), 1 / np.sqrt(2), 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, -1 / np.sqrt(2), 1.j / np.sqrt(2), 0],
         [0, 1.j / np.sqrt(2), -1 / np.sqrt(2), 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 1.j / np.sqrt(2), -1 / np.sqrt(2), 0],
         [0, 0, 0, 1.j, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1]]

Ujump = [[1, 0, 0, 0],
         [0, 0, 1.j, 0],
         [0, 1.j, 0, 0],
         [0, 0, 0, 1]]
         
Umerge=np.matrix(Usplit).getH()

#### Example of Split operation and then merge on a 3x3 board. This is a simple example to ensure that these gates do work correctly.

In [3]:
q2 = QuantumRegister(4, "box")
c2 = ClassicalRegister(4)
bell = QuantumCircuit(q2, c2)
bell.x(0)
target_state_bell = qi.Statevector.from_instruction(bell.reverse_bits()).to_dict()
print(target_state_bell)

bell.unitary(Usplit,[0,1,2])
bell.draw()

{'1000': (1+0j)}


┌───┐┌──────────┐
box_0: ┤ X ├┤0         ├
       └───┘│          │
box_1: ─────┤1 Unitary ├
            │          │
box_2: ─────┤2         ├
            └──────────┘
box_3: ─────────────────
                        
 c0: 4/═════════════════

In [4]:
target_state_bell = qi.Statevector.from_instruction(bell.reverse_bits()).to_dict()
print(target_state_bell)

{'0010': 0.7071067811865475j, '0100': 0.7071067811865475j}


In [5]:
bell.unitary(Umerge,[3,2,1])
bell.draw()

┌───┐┌──────────┐            
box_0: ┤ X ├┤0         ├────────────
       └───┘│          │┌──────────┐
box_1: ─────┤1 Unitary ├┤2         ├
            │          ││          │
box_2: ─────┤2         ├┤1 Unitary ├
            └──────────┘│          │
box_3: ─────────────────┤0         ├
                        └──────────┘
 c0: 4/═════════════════════════════

In [6]:
# target_state_bell = qi.Statevector.from_instruction(bell)
target_state_bell = qi.Statevector.from_instruction(bell.reverse_bits()).to_dict()
print(target_state_bell)

{'0001': (0.9999999999999998+0j), '0010': -2.2371143170757382e-17j}


In [7]:
bell.measure([0,1,2,3], [3,2,1,0])
from qiskit import execute, Aer
job = execute(bell,Aer.get_backend('qasm_simulator'),shots=1024)
counts = job.result().get_counts(bell)
print(counts) # counts is a dictionary

{'0001': 1024}


### State of the Board:

The state of the board is being created here.

The state of the board consists of (number of boxes * 2) qubits, meaning each box in the grid is represented in 2 qubits. The first qubit represents the occupancy of the box, 1 in case of being occupied and 0 if not occupied. The second qubit will represent the color of the piece in it, 0 if white and 1 if black.

The state will store every alternate box only since every other alternate box is unused in the game of checkers, i.e the piece cannot move there.

The game will simulate a 6x6 checkerboard meaning 36 qubits will be stored in the state.


In [8]:
import pprint

board = []
c_board_regs = []
pos_dic = {}   

box_to_qubit = []

white_occ = [(0,0), (0,2), (0, 4), (1,1), (1,3), (1, 5)]
black_occ = [(4,1), (4,3), (4, 5), (5,0), (5,2), (5, 4)]

remove = [(1,0), (3,0), (5,0), (0,1), (2,1), (4,1), (1,2), (3,2), (5,2), (0,3), (2,3), (4,3), (1,4), (3,4), (5,4), (0,5), (2,5), (4,5)]

def init():
    
    for i in range(6):
        
        temp = []

        for j in range(6):

            pos_dic[f'box_({i},{j})'] = i, j
            
            if (j,i) not in remove:
                qreg = QuantumRegister(2, name=f'box_({j},{i})')
                c_board_regs.append(ClassicalRegister(2, name=f'cbox_({j},{i})'))
                box_to_qubit.append((j,i))
                box_to_qubit.append((j,i))

                temp.append(qreg)       
        board.append(temp)

init()

temp = []
for i in board:
    temp += i
   
# print(temp)
q_board = QuantumCircuit(*temp, *c_board_regs)


for i in range(0, 12, 2):
    q_board.x(i)

for i in range(35, 23, -2):
    q_board.x(i)
    q_board.x(i-1)
    

q_board.barrier()

q_board.draw()



┌───┐ ░ 
 box_(0,0)_0: ┤ X ├─░─
              └───┘ ░ 
 box_(0,0)_1: ──────░─
              ┌───┐ ░ 
 box_(2,0)_0: ┤ X ├─░─
              └───┘ ░ 
 box_(2,0)_1: ──────░─
              ┌───┐ ░ 
 box_(4,0)_0: ┤ X ├─░─
              └───┘ ░ 
 box_(4,0)_1: ──────░─
              ┌───┐ ░ 
 box_(1,1)_0: ┤ X ├─░─
              └───┘ ░ 
 box_(1,1)_1: ──────░─
              ┌───┐ ░ 
 box_(3,1)_0: ┤ X ├─░─
              └───┘ ░ 
 box_(3,1)_1: ──────░─
              ┌───┐ ░ 
 box_(5,1)_0: ┤ X ├─░─
              └───┘ ░ 
 box_(5,1)_1: ──────░─
                    ░ 
 box_(0,2)_0: ──────░─
                    ░ 
 box_(0,2)_1: ──────░─
                    ░ 
 box_(2,2)_0: ──────░─
                    ░ 
 box_(2,2)_1: ──────░─
                    ░ 
 box_(4,2)_0: ──────░─
                    ░ 
 box_(4,2)_1: ──────░─
                    ░ 
 box_(1,3)_0: ──────░─
                    ░ 
 box_(1,3)_1: ──────░─
                    ░ 
 box_(3,3)_0: ──────░─
                    ░ 
 box_(3,3)_1: ──────░─
                    ░ 
 box_(5,3)_0: ──────░─
                    ░ 
 box_(5,3)_1: ──────░─
              ┌───┐ ░ 
 box_(0,4)_0: ┤ X ├─░─
              ├───┤ ░ 
 box_(0,4)_1: ┤ X ├─░─
              ├───┤ ░ 
 box_(2,4)_0: ┤ X ├─░─
              ├───┤ ░ 
 box_(2,4)_1: ┤ X ├─░─
              ├───┤ ░ 
 box_(4,4)_0: ┤ X ├─░─
              ├───┤ ░ 
 box_(4,4)_1: ┤ X ├─░─
              ├───┤ ░ 
 box_(1,5)_0: ┤ X ├─░─
              ├───┤ ░ 
 box_(1,5)_1: ┤ X ├─░─
              ├───┤ ░ 
 box_(3,5)_0: ┤ X ├─░─
              ├───┤ ░ 
 box_(3,5)_1: ┤ X ├─░─
              ├───┤ ░ 
 box_(5,5)_0: ┤ X ├─░─
              ├───┤ ░ 
 box_(5,5)_1: ┤ X ├─░─
              └───┘ ░ 
cbox_(0,0): 2/════════
                      
cbox_(2,0): 2/════════
                      
cbox_(4,0): 2/════════
                      
cbox_(1,1): 2/════════
                      
cbox_(3,1): 2/════════
                      
cbox_(5,1): 2/════════
                      
cbox_(0,2): 2/════════
                      
cbox_(2,2): 2/════════
                      
cbox_(4,2): 2/════════
                      
cbox_(1,3): 2/════════
                      
cbox_(3,3): 2/════════
                      
cbox_(5,3): 2/════════
                      
cbox_(0,4): 2/════════
                      
cbox_(2,4): 2/════════
                      
cbox_(4,4): 2/════════
                      
cbox_(1,5): 2/════════
                      
cbox_(3,5): 2/════════
                      
cbox_(5,5): 2/════════

In [9]:
pos_dic = {}
# lst_x = ["a", "b", "c", "d", "e", "f"]
# lst_y = ["0", "1", "2", "3", "4", "5"]

for i in range(6):
    for j in range(6):
        pos_dic[f'box_({i},{j})'] = i, j



### Gates

We have three unitaries for the operations of the jump, split jump and merge. 

The jump unitary is a swap operator that acts on two inputs of two qubits each. There is the target box and the source box each of which holds two qubits to describe its state completely. We apply our jump unitary to swap the states of the target and source.

The split unitary works on three inputs/boxes. There are two target boxes and one source box each of which is composed of 2 qubits each. It allows the player to move their piece into a superposition. The split unitary removes the occupancy and color qubit of the source box and creates two superpositions in the state, each of which contains the move in a single direction. For e.g: one superposition will contain the state if the piece jumped in the left diagonal and one superposition will contain the state if the piece jumped in the right diagonal.

The merge unitary works on three inputs/boxes as well. There are two source boxes and one target box. It is the dagger of the split unitary. It allows the player to transfer their superposition piece back into classical state. It sets the occupancy and color qubit of the source boxes to 0 and sets the occupancy qubit of target box to 1 and color qubit accordingly.

## Moves
- Jump Unitary


    ![jump](jump.png)



$\newline$
- Split


     ![split](split.png)




$\newline$
- Merge


    ![merge](merge.png)

In [10]:
def jump(src, target, qc):
    for i in range(2):
        qc.unitary(Ujump, [src[i], target[i]])

def split(src, target1, target2, qc):
    # for i in range(2):
    #     qc.unitary(Usplit, [src[i], target1[i], target2[i]])
    # qc.unitary(Usplit, [src[1], target1[1], target2[1]])
    qc.unitary(Usplit, [src[0], target1[0], target2[0]])

def merge(src1, src2, target, qc):
    qc.unitary(Umerge, [src1[0], src2[0], target[0]])
    qc.unitary(Umerge, [src1[1], src2[1], target[1]])




In [11]:
def get_diag_white(box):
    return (pos_dic[box][0]+1, pos_dic[box][1]+1), (pos_dic[box][0]-1, pos_dic[box][1]+1)  

def get_diag_black(box):
    return (pos_dic[box][0]+1, pos_dic[box][1]-1), (pos_dic[box][0]-1, pos_dic[box][1]-1)  

 
src = 'box_(1,1)'
diag1, diag2 = get_diag_white('box_(1,1)')

# print(diag1, diag2)
# print(diag1)
# print(box_to_qubit.index(diag1))
src_bits = (box_to_qubit.index((1,1)), box_to_qubit.index((1,1))+1)
target1_bits = (box_to_qubit.index(diag1), box_to_qubit.index(diag1) + 1)
target2_bits = (box_to_qubit.index(diag2), box_to_qubit.index(diag2) + 1)

# jump(src_bits, target1_bits, q_board)
# split(src_bits, target1_bits, target2_bits, q_board)


src = 'box_(4,4)'
diag1, diag2 = get_diag_white('box_(4,4)')

src_bits = (box_to_qubit.index((4,4)), box_to_qubit.index((4,4))+1)
target1_bits = (box_to_qubit.index(diag1), box_to_qubit.index(diag1) + 1)
target2_bits = (box_to_qubit.index(diag2), box_to_qubit.index(diag2) + 1)

jump(src_bits, target1_bits, q_board)
# split(src_bits, target1_bits, target2_bits, q_board)
# merge(target1_bits, target2_bits, src_bits, q_board)
#qi.Statevector.from_instruction(q_board.reverse_bits()).to_dict()


q_board.measure([i for i in range(36)], [i for i in range(35, -1, -1)])
job = execute(q_board,Aer.get_backend('qasm_simulator'),shots=256)
counts = job.result().get_counts(q_board)
print(counts) # counts is a dictionary


{'10 10 10 10 10 10 00 00 00 00 00 00 11 11 11 11 11 11': 256}


## Updated split jump with pathing consideration

We apply a different type of Unitary called the slide unitary:

 ![jump](split_slide.png)


## Simple Classical Capture 

We need to know the diagonals upto to two levels of boxes and then we apply the following matrix:


In [12]:
Cap=[[1,0,0,0,0,0,0,0],
     [0,0,0,0,1,0,0,0],
     [0,0,0,0,0,1,0,0],
     [0,0,0,0,0,0,1,0],
     [0,1,0,0,0,0,0,0],
     [0,0,1,0,0,0,0,0],
     [0,0,0,1,0,0,0,0],
     [0,0,0,0,0,0,0,1]]

Capdag=np.matrix(Cap).getH()

result= np.dot(Cap,Capdag)
print(result)

def updateColSplit(t1,t2,s,ancila):
     bell.cnot(s,ancila)
     bell.swap(s,t1)
     bell.swap(ancila,t2)

def updateCColSplit(to1,to2,t1,t2,s,ancila):
     # bell.cnot(s,ancila)
     bell.swap(ancila,s)
     bell.cswap(to1,ancila,t1)
     bell.cswap(to2,ancila,t2)
     

     

def updateColcapture(t,p,s,ancila):
     bell.cnot(p,p)
     bell.swap(s,t)


[[1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1]]


In [13]:
def split_jump_updated(t1,t2,s,ancila, qc):
    qc.unitary(Usplit,[s, t1, t2])

    updateCColSplit(t1+13, t2+13, t1, t2, s, ancila)

    

In [14]:
from qiskit import execute, Aer
qcap = QuantumRegister(27, "box")
ccap = ClassicalRegister(27)
bell = QuantumCircuit(qcap, ccap)
## Setting states

sppoky = ["*", ".", "/", ":", "!", "^", "-", "~"]

## Setting bottom left box to have black piece


## Setting the center box to have a white piece
bell.x(0)
bell.x(1)
bell.x(2)
bell.x(3)
bell.x(4)


bell.x(8)
bell.x(9)
bell.x(10)
bell.x(11)
bell.x(12)
bell.x(8+13)
bell.x(9+13)
bell.x(10+13)
bell.x(11+13)
bell.x(12+13)

#bell.unitary(Cap,[0,8,4])
# bell.unitary(Cap,[1,9,5])



# bell.unitary(Usplit,[9,7,6])
#updateColSplit(t1=6+13,t2=7+13,s=9+13,ancila=26)
# updateCColSplit(to1=6,to2=7,t1=6+13,t2=7+13,s=9+13,ancila=26)

# bell.measure(list(range(27)), list(range(26,-1,-1)))


# job = execute(bell,Aer.get_backend('qasm_simulator'),shots=20)
# counts = job.result().get_counts(bell)
# print(counts) # counts is a dictionary
counts = {'111110001111100000000111110': 20}
# bell.draw()

def print_state_board2(counts, size):
    state = []

    counts_temp = {}

    for i in counts:
        i_ = i.replace(" ", "")
        counts_temp[i_] = counts[i]

    counts = counts_temp
    
    count = 0
    count_alt = False
    row = ""

    # print(counts)

    for i in range(0, 13):

        # i = i//2

        box = " "
        
        for k in counts:
            if k[i] == '1':
                if k[i+13] == '1':
                    box = "B"
                else:
                    box = "W"
        
        row += box + '\t\t'

        count+=1

        if (count == 3 and not count_alt) or (count == 2 and count_alt):
            count = 0
            if count_alt:
                state.append("\t" + row[:-2])
            else:
                state.append(row[:-2])

            count_alt = not count_alt
            # print(row[0])
            row = ""


    state = state[::-1]

    output = ""
    
    for i in state[:-1]:
        # print(i)
        # print("-----------------" * (size//2))
        output += i + "\n"
        output += ("-----------------" * (size//2)) + "\n"
    
    # print(state[-1])
    output += state[-1] + "\n"

    print(output)



print_state_board2(counts, 5)
print("\n\n\n")

input_ = ""
s = 0
t = 0
split_jump = False

while s != -2 and t!= -2:
    s = int(input("Enter Source Box"))

    if s==-1:
        bell.measure(list(range(27)), list(range(26,-1,-1)))


        job = execute(bell,Aer.get_backend('qasm_simulator'),shots=1)
        counts = job.result().get_counts(bell)

        print_state_board2(counts, 5)
        print("\n\n\n")

        s = int(input("Enter Source Box"))

    t = input("Enter Target Box (Spaced if split jump)").split()
    
    if len(t) == 2:
        split_jump = True
        t1, t2 = int(t[0]), int(t[1])
    else:
        t1 = int(t[0])
        split_jump = False

    for i in list(counts.keys())[0:1]:
        temp = list(i)

        temp[t1] = temp[s]
        temp[t1+13] = temp[s+13]

        if split_jump:
            temp[t2] = temp[s]
            temp[t2+13] = temp[s+13]

            split_jump_updated(t1, t2, s, 26, bell)
            

        temp[s] = '0'
        temp[s+13] = '0'
        # print(temp)
        temp = "".join(temp)

        counts[temp] = counts[i]

        del counts[i]
    print_state_board2(counts, 5)
    print("\n\n\n")

# print("\r", end="")
# print("asd")
# print_state_board2(counts, 5)
# print ("\033[Aghfghfghfhfgh\033[A")
# print ("\033[Aghfghfghfhfgh\033[A")


B		B		B
----------------------------------
	B		B
----------------------------------
 		 		 
----------------------------------
	W		W
----------------------------------
W		W		W

['1', '1', '1', '0', '1', '1', '1', '0', '1', '1', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '1', '1', '0']
B		B		B
----------------------------------
	B		B
----------------------------------
W		W		 
----------------------------------
	 		W
----------------------------------
W		W		W





['1', '1', '1', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '1', '1', '0']
B		B		B
----------------------------------
	B		B
----------------------------------
W		W		W
----------------------------------
	 		 
----------------------------------
W		W		W





B		B		B
----------------------------------
	B		B
----------------------------------
 		W		W
----------------------------------
	 		 
----------------------------------
W		W		W







IndexError: list index out of range

In [ ]:
from qiskit import execute, Aer
qcap = QuantumRegister(2, "box")
ccap = ClassicalRegister(2)
bell = QuantumCircuit(qcap, ccap)

bell.x(0)
bell.cnot(0,1)
bell.cnot(1,0)

bell.draw()

bell.measure([0,1],[1,0])
job = execute(bell,Aer.get_backend('qasm_simulator'),shots=32)
counts = job.result().get_counts(bell)
print(counts)

{'01': 32}
